In [ ]:
# Uncomment line below to install exlib
# !pip install exlib

In [5]:
import torch
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
import numpy as np
import pandas as pd
import tqdm
from tqdm import tqdm
import torch.nn as nn
from torch.utils.data import DataLoader
from datasets import load_dataset
import sentence_transformers

# import exlib
# from exlib.datasets.multilingual_politeness import PolitenessDataset, PolitenessClassifier, Metric, get_politeness_scores
# from exlib.utils.politeness_helper import load_lexica

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ModuleNotFoundError: No module named 'exlib'

## Load datasets and pre-trained model

In [ ]:
import exlib
from exlib.datasets.multilingual_politeness import PolitenessDataset, PolitenessClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Sample inference on dataset

In [45]:
dataset = PolitenessDataset("train")
dataloader = DataLoader(dataset, batch_size=4, shuffle=False)
model = PolitenessClassifier()
model.to(device)
model.eval()

for batch in tqdm(dataloader): 
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    utterances = [dataset.tokenizer.decode(input_id, skip_special_tokens=True) for input_id in input_ids]
    for utterance, label in zip(utterances, output):
        print("Text: {}\nPoliteness: {}\n".format(utterance, label.item()))
    break

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at BrachioLab/xlm-roberta-politeness and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/1140 [00:00<?, ?it/s]

Text: That is why he is a ‘pretender’. He has never claimed to be a King - or a Kaiser, for that matter. He is in the same class as the Comte de Paris, who is not the King of France, but would be if the Bourbons were placed on a restored French throne.
Politeness: -0.07154521346092224

Text: Let's knock any 'EngVar' shenanigans on the head right away, shall we? The Manual of Style, as I understand it, makes it clear that the subject's national ties and own language set the course.
Politeness: -0.12842649221420288

Text: Thank you for your contributions. There are some conventions that apply to articles, and medical articles in particular. Secondary sources were available for the material, and should be cited to validate the medical information from the studies.
Politeness: 0.11653748154640198

Text: The conversion of tacit to explicit knowledge is seen in for example the bread making machine's case. In response to your question, culture is a broad term. I would like to narrow it down t

### load test data + model

In [4]:
dataset = PolitenessDataset("test")
model = PolitenessClassifier()

model.to(device)
model.eval()

NameError: name 'PolitenessDataset' is not defined

## Define Alignment Metric

In [ ]:
from exlib.datasets.multilingual_politeness import Metric

metric = Metric()
# we define this is the get_politeness_scores function below, so this is technically redundant

## Baselines

In [ ]:
from exlib.datasets.multilingual_politeness import get_politeness_scores

In [ ]:
all_baselines_scores = get_politeness_scores(baselines = ['word', 'phrase', 'sentence'])

In [ ]:
for name in all_baselines_scores:
    metric = torch.tensor(all_baselines_scores[name])
    mean_metric = metric.nanmean()
    print(f'BASELINE {name} mean score: {mean_metric}')

### BERTopic (Clustering)

create topics from the dataset

In [59]:
# first, save all the utterances

dataset = PolitenessDataset("test")
utterances = [' '.join(dataset[i]['word_list']) for i in range(len(dataset))]
torch.save(utterances, '../../fix/utterances/multilingual_politeness_test.pt')

In [6]:
# then, install bertopic + use them on the utterances

!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 77.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 88.2 MB/s eta 0:00:0000:0100:01


In [ ]:
all_baselines_scores = get_politeness_scoresget_politeness_scores(baselines = ['clustering'])

In [ ]:
for name in all_baselines_scores:
    metric = torch.tensor(all_baselines_scores[name])
    mean_metric = metric.nanmean()
    print(f'BASELINE {name} mean score: {mean_metric}')